In [12]:
# --- Governance_Fairness.ipynb : CELL 1
import pandas as pd
import numpy as np
import joblib

# 1) Load models
model_young = joblib.load('artifacts/model_young.joblib')   # age ≤ 25 (LinearRegression)
model_rest  = joblib.load('artifacts/model_rest.joblib')    # age > 25 (XGBRegressor)

# 2) Load scaler bundles (each is a dict with {'scaler': transformer, 'cols_to_scale': [...]})
bundle_young = joblib.load('artifacts/scaler_young.joblib')
bundle_rest  = joblib.load('artifacts/scaler_rest.joblib')

scaler_young = bundle_young['scaler']
cols_scale_y = list(bundle_young['cols_to_scale'])

scaler_rest  = bundle_rest['scaler']
cols_scale_r = list(bundle_rest['cols_to_scale'])

# 3) Load unified test set: must include ALL features you used + target 'premium'
test_df = pd.read_csv('test_data.csv')

# Basic checks
assert 'age' in test_df.columns, "test_data.csv must contain 'age' (for model routing)."
assert 'annual_premium_amount' in test_df.columns, "test_data.csv must contain the target column 'annual_premium_amount'."

# Show quick shape
test_df.shape


(6026, 19)

In [15]:
# --- Governance_Fairness.ipynb : CELL 2 (updated)

# Use exact training-time feature order captured in the model artifacts
feat_cols_young = list(getattr(model_young, 'feature_names_in_', []))
feat_cols_rest  = [str(c) for c in getattr(model_rest,  'feature_names_in_', [])]  # cast np.str_ → str

if not feat_cols_young or not feat_cols_rest:
    raise ValueError("Models do not expose feature_names_in_. Save & load training column lists.")

# Check nothing missing for each segment
missing_y = [c for c in feat_cols_young if c not in test_df.columns]
missing_r = [c for c in feat_cols_rest  if c not in test_df.columns]
if missing_y or missing_r:
    raise ValueError(f"Missing columns — YOUNG: {missing_y} | REST: {missing_r}")

# Routing masks
mask_y = test_df['age'] <= 25
mask_r = test_df['age'] > 25

# Segment matrices in the exact trained order
X_y = test_df.loc[mask_y, feat_cols_young].copy()
X_r = test_df.loc[mask_r,  feat_cols_rest ].copy()

def scale_with_expected(X_in: pd.DataFrame, scaler, cols_expected: list) -> pd.DataFrame:
    """
    Safe scaling:
      - If X_in is empty, return it unchanged (avoids MinMaxScaler error).
      - Build a temp DF with the scaler's expected schema; fill any missing
        expected cols with a neutral value (training stats if available, else 0).
      - Transform, then write back scaled values only for overlapping cols.
    """
    if X_in.shape[0] == 0:
        return X_in  # <-- critical fix: no transform on empty input

    X = X_in.copy()
    original_cols = list(X.columns)

    # temp DF with full expected schema
    temp = pd.DataFrame(index=X.index)
    for i, col in enumerate(cols_expected):
        if col in X.columns:
            temp[col] = X[col]
        else:
            # Neutral filler: try to use scaler stats; else 0.0
            filler = 0.0
            # StandardScaler: mean_
            if hasattr(scaler, "mean_"):
                filler = float(scaler.mean_[i])
            # MinMaxScaler: data_min_ / data_max_ -> mid-point is neutral-ish
            elif hasattr(scaler, "data_min_") and hasattr(scaler, "data_max_"):
                mn = float(scaler.data_min_[i]); mx = float(scaler.data_max_[i])
                filler = (mn + mx) / 2.0
            temp[col] = filler

    # transform with exact expected schema/order
    Xt = scaler.transform(temp[cols_expected])
    Xt = pd.DataFrame(Xt, index=temp.index, columns=cols_expected)

    # write back only overlapping cols (preserve original order/set)
    cols_to_write = [c for c in cols_expected if c in original_cols]
    if cols_to_write:
        X.loc[:, cols_to_write] = Xt[cols_to_write]

    return X[original_cols]

# Apply segment-specific scaling safely (now empty-safe)
X_y_scaled = scale_with_expected(X_y, scaler_young, cols_scale_y)
X_r_scaled = scale_with_expected(X_r, scaler_rest,  cols_scale_r)

# Predict (already empty-safe)
y_pred = pd.Series(index=test_df.index, dtype=float)
if X_y_scaled.shape[0] > 0:
    y_pred.loc[X_y_scaled.index] = model_young.predict(X_y_scaled)
if X_r_scaled.shape[0] > 0:
    y_pred.loc[X_r_scaled.index] = model_rest.predict(X_r_scaled)

# Assemble df_predictions
df_predictions = test_df.copy()
df_predictions['y_true'] = df_predictions['annual_premium_amount']
df_predictions['y_pred'] = y_pred
df_predictions['model_used'] = np.where(mask_r, 'Model A (>25)', 'Model B (≤25)')
df_predictions['model_version'] = df_predictions['model_used'].map({
    'Model A (>25)': 'v1.0-XGB',
    'Model B (≤25)': 'v1.0-LR'
})

print("Rows routed → young (<=25):", X_y.shape[0], "| rest (>25):", X_r.shape[0])
print(df_predictions.shape)
df_predictions.head()


Rows routed → young (<=25): 6026 | rest (>25): 0
(6026, 23)


,age,number_of_dependants,income_lakhs,insurance_plan,genetical_risk,normalized_risk_score,gender_Male,region_Northwest,region_Southeast,region_Southwest,...,bmi_category_Underweight,smoking_status_Occasional,smoking_status_Regular,employment_status_Salaried,employment_status_Self-Employed,annual_premium_amount,y_true,y_pred,model_used,model_version
0,0.428571,1.0,0.080808,0.0,0.2,0.428571,1,0,0,0,...,0,0,0,0,0,5452,5452,1172.965391,Model B (≤25),v1.0-LR
1,0.714286,0.0,0.030303,0.0,0.8,1.000000,1,0,0,1,...,0,0,0,0,0,9658,9658,2433.214053,Model B (≤25),v1.0-LR
2,0.142857,0.0,0.363636,0.0,0.0,1.000000,1,0,0,1,...,0,0,1,1,0,6031,6031,2250.253142,Model B (≤25),v1.0-LR
3,0.428571,0.0,0.515152,0.5,0.8,0.428571,1,0,1,0,...,0,0,0,0,0,12386,12386,3933.896263,Model B (≤25),v1.0-LR
4,0.000000,0.0,0.222222,0.0,0.0,0.000000,0,0,1,0,...,1,0,1,0,0,4245,4245,1213.435223,Model B (≤25),v1.0-LR


In [19]:
# --- Rebuild categorical columns for fairness from one-hot columns

import numpy as np
import pandas as pd

def onehot_to_label(df: pd.DataFrame, base: str, unknown_label: str = "Unknown") -> pd.Series:
    """
    Collapse one-hot columns like base_* back into a single categorical label Series.
    Example: base='smoking_status' -> looks for columns starting with 'smoking_status_'.
    If a row has all zeros (or columns missing), returns 'Unknown'.
    """
    cols = [c for c in df.columns if c.startswith(base + "_")]
    if not cols:
        # No one-hots present for this base
        return pd.Series([unknown_label] * len(df), index=df.index, name=f"{base}_grp")

    sub = df[cols].copy()

    # Handle any non-binary due to float quirks by thresholding at 0.5
    sub = (sub.values > 0.5).astype(int)
    sub = pd.DataFrame(sub, index=df.index, columns=cols)

    # Argmax to pick the active category per row
    idx = sub.values.argmax(axis=1)
    labels = [cols[i].replace(base + "_", "") for i in idx]

    # If a row has no active one-hot (all zeros), mark Unknown
    none_active = sub.sum(axis=1).values == 0
    if none_active.any():
        for i, na in enumerate(none_active):
            if na:
                labels[i] = unknown_label

    return pd.Series(labels, index=df.index, name=f"{base}_grp")

# Rebuild groups we want to audit
df_fair = df_predictions.copy()

# Gender, Smoking, BMI category, Region from one-hots
df_fair['gender_grp']          = onehot_to_label(df_fair, 'gender')
df_fair['smoking_status_grp']  = onehot_to_label(df_fair, 'smoking_status')
df_fair['bmi_category_grp']    = onehot_to_label(df_fair, 'bmi_category')
df_fair['region_grp']          = onehot_to_label(df_fair, 'region')

# Income level: try one-hot first; if not present, derive from income_lakhs
if any(col.startswith('income_level_') for col in df_fair.columns):
    df_fair['income_level_grp'] = onehot_to_label(df_fair, 'income_level')
else:
    # Derive from income_lakhs into your four buckets: <10L, 10L - 25L, 25L - 40L, > 40L
    def derive_income_level(x):
        try:
            v = float(x)
        except Exception:
            return "Unknown"
        if v < 10:
            return "<10L"
        elif v < 25:
            return "10L - 25L"
        elif v < 40:
            return "25L - 40L"
        else:
            return "> 40L"

    if 'income_lakhs' in df_fair.columns:
        df_fair['income_level_grp'] = df_fair['income_lakhs'].apply(derive_income_level)
    else:
        df_fair['income_level_grp'] = "Unknown"  # fallback

# Optional: clean up common typos (e.g., Occassional → Occasional)
def fix_label_typos(s: pd.Series) -> pd.Series:
    mapping = {
        "Occassional": "Occasional",  # common misspelling
        "Northwest ": "Northwest",    # stray spaces, if any
        "Southwest ": "Southwest",
    }
    return s.replace(mapping)

df_fair['smoking_status_grp'] = fix_label_typos(df_fair['smoking_status_grp'])
df_fair['region_grp']         = fix_label_typos(df_fair['region_grp'])


In [20]:
# --- Fairness metrics on rebuilt group columns

def fairness_metrics(df: pd.DataFrame, group_col: str, tolerance: float = 0.10):
    """
    df must contain: y_true, y_pred, and group_col
    tolerance = percentage window to count a prediction as 'affordable' (±10% default)
    Returns: (metrics_df, overall_mae, overall_affordable_rate)
    """
    use = df[['y_true','y_pred', group_col]].dropna().copy()
    use['err'] = use['y_pred'] - use['y_true']
    use['within_tol'] = (use['err'].abs() <= tolerance * use['y_true']).astype(int)

    overall_mae = use['err'].abs().mean()
    overall_aff = use['within_tol'].mean()

    metrics = (use
        .groupby(group_col, dropna=False)
        .agg(
            count=('y_true', 'size'),
            MAE=('err', lambda x: x.abs().mean()),
            Overcharge=('err', lambda x: np.maximum(x, 0).mean()),
            Undercharge=('err', lambda x: np.minimum(x, 0).mean()),
            Affordable_Pct=('within_tol', 'mean')
        )
        .assign(
            DIR=lambda g: g['Affordable_Pct'] / (overall_aff if overall_aff > 0 else 1.0),
            MAE_vs_overall=lambda g: (g['MAE'] - overall_mae) / (overall_mae if overall_mae > 0 else 1.0)
        )
        .sort_values('MAE', ascending=False)
        .round(3)
    )
    return metrics, round(overall_mae, 3), round(overall_aff, 3)

attributes_to_check = [
    'gender_grp',
    'smoking_status_grp',
    'income_level_grp',
    'bmi_category_grp',
    'region_grp'
]

fairness_results = {attr: fairness_metrics(df_fair, attr) for attr in attributes_to_check}

# Quick peeks
fairness_results['gender_grp'][0], fairness_results['smoking_status_grp'][0]


(            count       MAE  Overcharge  Undercharge  Affordable_Pct  DIR  \
 gender_grp                                                                  
 Unknown      2686  6187.583         0.0    -6187.583             0.0  0.0   
 Male         3340  6101.189         0.0    -6101.189             0.0  0.0   
 
             MAE_vs_overall  
 gender_grp                  
 Unknown              0.008  
 Male                -0.006  ,
                     count       MAE  Overcharge  Undercharge  Affordable_Pct  \
 smoking_status_grp                                                             
 Unknown              4149  6147.945         0.0    -6147.945             0.0   
 Regular              1357  6139.435         0.0    -6139.435             0.0   
 Occasional            520  6074.581         0.0    -6074.581             0.0   
 
                     DIR  MAE_vs_overall  
 smoking_status_grp                       
 Unknown             0.0           0.001  
 Regular             0.0     